In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [5]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.AGD(
    model.parameters(),
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    model=model,
    use_diagonal=False,
    c1=1e-4,
    tau=0.5,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch] += loss

    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch - 1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience

    print(", loss: {}".format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.5174350738525391
epoch:  1, loss: 0.06672327220439911
epoch:  2, loss: 0.03232215344905853
epoch:  3, loss: 0.02538452297449112
epoch:  4, loss: 0.017865171656012535
epoch:  5, loss: 0.01703009195625782
epoch:  6, loss: 0.011467866599559784
epoch:  7, loss: 0.010719282552599907
epoch:  8, loss: 0.00985685270279646
epoch:  9, loss: 0.009250903502106667
epoch:  10, loss: 0.008748169057071209
epoch:  11, loss: 0.008593478240072727
epoch:  12, loss: 0.006469870917499065
epoch:  13, loss: 0.0058463201858103275
epoch:  14, loss: 0.0057974462397396564
epoch:  15, loss: 0.00461881747469306
epoch:  16, loss: 0.0043914406560361385
epoch:  17, loss: 0.003793660318478942
epoch:  18, loss: 0.003650564467534423
epoch:  19, loss: 0.002931748516857624
epoch:  20, loss: 0.002633826807141304
epoch:  21, loss: 0.002433792455121875
epoch:  22, loss: 0.0021935426630079746
epoch:  23, loss: 0.002015691017732024
epoch:  24, loss: 0.0019404221093282104
epoch:  25, loss: 0.001910668564960360

In [6]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.988854822157648
Test metrics:  R2 = 0.9868933637297932


In [7]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.AGD(
    model.parameters(),
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    model=model,
    use_diagonal=True,
    c1=1e-4,
    tau=0.1,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch] += loss

    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch - 1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience

    print(", loss: {}".format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.17654085159301758
epoch:  1, loss: 0.02821688912808895
epoch:  2, loss: 0.02365739829838276
epoch:  3, loss: 0.010598459281027317
epoch:  4, loss: 0.009633749723434448
epoch:  5, loss: 0.00860026478767395
epoch:  6, loss: 0.007722286973148584
epoch:  7, loss: 0.007204593624919653
epoch:  8, loss: 0.006601330824196339
epoch:  9, loss: 0.0061677913181483746
epoch:  10, loss: 0.005742161069065332
epoch:  11, loss: 0.005486208945512772
epoch:  12, loss: 0.005222318693995476
epoch:  13, loss: 0.0050088874995708466
epoch:  14, loss: 0.004778837785124779
epoch:  15, loss: 0.004520828370004892
epoch:  16, loss: 0.004188455641269684
epoch:  17, loss: 0.0038940904196351767
epoch:  18, loss: 0.0036144577898085117
epoch:  19, loss: 0.003386372933164239
epoch:  20, loss: 0.003162921406328678
epoch:  21, loss: 0.002991692628711462
epoch:  22, loss: 0.002902550157159567
epoch:  23, loss: 0.0021084703039377928
epoch:  24, loss: 0.0018627080135047436
epoch:  25, loss: 0.0017172977095

In [8]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.998035684483543
Test metrics:  R2 = 0.9973266055840903
